# Mesh generation for tutorial 07

In [ ]:
import pygmsh
from numpy import fabs, finfo, logical_and, logical_or, sqrt
from dolfinx import *
from dolfinx.cpp.mesh import CellType, GhostMode
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters

In [ ]:
r = 1
lcar = 2

### Create mesh

Generate a simple mesh consisting in an hexagon discretized with six equilateral triangle cells.

In [ ]:
geom = pygmsh.built_in.Geometry()
c0 = geom.add_circle([0.0, 0.0, 0.0], r, lcar, num_sections=6)
pygmsh_mesh = pygmsh.generate_mesh(geom)
mesh = Mesh(MPI.comm_world, CellType.triangle, pygmsh_mesh.points[:, :2], pygmsh_mesh.cells_dict["triangle"], [], GhostMode.none)

In [ ]:
plot(mesh)

### Create cell restrictions

Define a mesh function on cells, which is equal to one on all cells.

In [ ]:
cell_restriction_all = MeshFunction("size_t", mesh, mesh.topology.dim, 1)

Define a mesh function on cells, which is equal to zero on one half of the cells, and equal to one on the other half of the cells

In [ ]:
eps = finfo(float).eps
cell_restriction_subset = MeshFunction("size_t", mesh, mesh.topology.dim, 0)
cell_restriction_subset.mark(lambda x: logical_or(x[0] < eps, logical_and(x[1] < eps, x[0] < 0.5 + eps)), 1)

### Create facet restrictions

Define a mesh function on facets, which is equal to one on all facets

In [ ]:
facet_restriction_all = MeshFunction("size_t", mesh, mesh.topology.dim - 1, 1)

Define a mesh function on facets, which is equal to one on two facets, and equal to zero on the remaining facets

In [ ]:
facet_restriction_subset = MeshFunction("size_t", mesh, mesh.topology.dim - 1, 0)
facet_restriction_subset.mark(lambda x: fabs(x[1] + sqrt(3)*x[0]) < 0.01, 1)

### Save

In [ ]:
with XDMFFile(MPI.comm_world, "hexagon.xdmf") as output:
    output.write(mesh)
with XDMFFile(MPI.comm_world, "hexagon_cell_restriction_all.xdmf") as output:
    output.write(cell_restriction_all)
with XDMFFile(MPI.comm_world, "hexagon_cell_restriction_subset.xdmf") as output:
    output.write(cell_restriction_subset)
with XDMFFile(MPI.comm_world, "hexagon_facet_restriction_all.xdmf") as output:
    output.write(facet_restriction_all)
with XDMFFile(MPI.comm_world, "hexagon_facet_restriction_subset.xdmf") as output:
    output.write(facet_restriction_subset)